In [7]:
import pandas as pd
from midiutil import MIDIFile
import time
import re
from yappyChuck import Client

client = Client()

In [2]:
#let's set up midi stuff
track    = 0
channel  = 0
track_time     = 0    # In beats
duration = 1    # In beats
tempo    = 60   # In BPM
volume   = 62  # 0-127, as per the MIDI standard

In [2]:
enmi = pd.read_csv('../enmi.csv')
sessions = pd.read_csv('../sessions.csv')
annotations = pd.read_csv('../annotations.txt')

In [3]:
annotation_data = pd.merge(enmi,annotations, how='left', on='id')
annotation_data.head()

,Unnamed: 0,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,quoted_status.contributors,quoted_status.is_quote_status,quoted_status.retweet_count,quoted_status.favorite_count,quoted_status.favorited,quoted_status.retweeted,quoted_status.lang,ann,start,end
0,0,Wed Dec 19 13:30:12 +0000 2018,1075382796179386369,1075382796179386369,"@robywebo @OdileA Bonjour, je parlais du panel...",False,"<a href=""http://twitter.com/download/iphone"" r...",1.075374e+18,1.075374e+18,1.828022e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454000,454000
1,1,Wed Dec 19 13:33:53 +0000 2018,1075383722323689472,1075383722323689472,#enmi18 @vincentpuig ou se trouve ces outils? ...,False,"<a href=""https://polemictweet.com"" rel=""nofoll...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,675000,675000
2,2,Wed Dec 19 13:34:23 +0000 2018,1075383846185635840,1075383846185635840,#enmi18 Paolo Vignola et Sara Baranzoni : inve...,False,"<a href=""https://polemictweet.com"" rel=""nofoll...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REF,705000,705000
3,3,Wed Dec 19 13:40:59 +0000 2018,1075385508635783170,1075385508635783170,Dernier session des #enmi18 : Le terrain du te...,True,"<a href=""https://about.twitter.com/products/tw...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REF,1101000,1101000
4,4,Wed Dec 19 13:45:22 +0000 2018,1075386610961784837,1075386610961784837,@Isabell42560134 #enmi18 Suivez les liens vers...,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",1.075384e+18,1.075384e+18,1.070035e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REF,1364000,1364000


In [4]:
def find_start_end(session):
    notes = annotation_data[(annotation_data['start'] >= session[0])&(annotation_data['end'] < session[1])]
    return notes[['text', 'user.screen_name', 'source', 'start']]

tweets = [find_start_end(session) for session in sessions.to_numpy()]

In [5]:
#clean html from soure field
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

### Source Code Reading

Using the source text to hear the network that creates the possibility of this network. 

In [ ]:
#@todo: place a source

for t in tweets:

    for tweet in t.to_numpy():
        
        #find the source to alter the rhythm. 
        source = cleanhtml(tweet[2])
        
        _source = ""
        if "Android" in source or "iOS" in source:
            _source = "source:{}:{}:{}".format(160.34, 0.15, 0.4)
        else:
            _source = "source:{}:{}:{}".format(230.34, 0.25, 0.4)

        msg = _source
        client.send(msg)
        time.sleep(.4)
        
        msg = "source:{}:{}:{}".format(63.4, 0.4, 0.5)
        client.send(msg)

        time.sleep(2)

## Types of Network within Network

This looks at the types of networks created within the network. 
How does Twitter allow us to create links between knowledge source, such as URLs. There is a separate argument about the type of URL but that is not dealt with here. The only one here is twitter as a way of allowing discourse. It takes a software-base approach to the discourse network and wonders what types of discourse comes from this discourse network. 

In [ ]:
for t in tweets:
    
    for tweet in t.to_numpy():
        search = t[0]
        #find urls
        match = re.findall(r'(http|ftp|https):\/\/([\w\-_]+(?:(?:\.[\w\-_]+)+))([\w\-\.,@?^=%&:/~\+#]*[\w\-\@?^=%&/~\+#])?', search)
        if match:
            for m in match:
                #very much to do but link url to a type
                if m[1] == "t.co":
                    client.send("url:{}:{}:{}".format(10, 0.25, volume))
                    client.send("url:{}:{}:{}".format(20, 0.25, volume))
                else:
                    client.send("url:{}:{}:{}".format(20, 0.25, volume))

        match = re.findall(r"#(\w+)", search)
        if match:
            for m in match:
                client.send("hashtag:{}:{}:{}".format(30, 0.25, volume))

        match = re.findall(r"@(\w+)", search)
        if match:
            for m in match:
                client.send("username:{}:{}:{}".format(40, 0.25, volume))

        time.sleep(2)